In [1]:
%matplotlib inline

In [87]:
import glob
from astropy.table import Table, Column
from astropy.io import fits
from collections import Counter
from __future__ import absolute_import, division, print_function

import os
from collections import Counter
import numpy as np
from astropy.io import fits
from astropy.table import Table, Column

import astropy.constants
c = astropy.constants.c.to('km/s').value

from desitarget.targets import desi_mask
from desisim.quickcat import quickcat

In [88]:
tile_names = glob.glob("/home/forero/Data/desitest/lowfat_serial_perfect/0/fiberassign/tile_*.fits")
nobs = Counter()
for tile_file in tile_names:
    fibassign = Table.read(tile_file, hdu=1)
    ii = (fibassign['TARGETID'] != -1) 
    nobs.update(fibassign['TARGETID'][ii])
    
targets = Table.read("/home/forero/Data/desitest/mtl/targets.fits")
truth = Table.read("/home/forero/Data/desitest/mtl/truth.fits")
mtl = Table.read("/home/forero/Data/desitest/lowfat_serial_perfect/0/mtl.fits")

In [ ]:
ids_fiber = list(nobs.keys())
ids_mtl = list(mtl['TARGETID'])
ids_truth = list(truth['TARGETID'])
ids_targets = list(targets['TARGETID'])

In [85]:
def find_missing(list_A, list_B): # |list_A| > |list_B|
    a = list(set(list_A) & set(list_B)) #find the intersection
    missing = list(set(list_B) - set(a)) #find B minus the intersection
    return missing #these are the IDs in B that are missing in A

In [86]:
missing = find_missing(ids_mtl, ids_truth)
print(len(missing))
missing = find_missing(ids_targets, ids_mtl)
print(len(missing))
missing = find_missing(ids_targets, ids_fiber)
print(len(missing))
missing = find_missing(ids_mtl, ids_fiber)
print(len(missing))
missing = find_missing(ids_truth, ids_fiber)
print(len(missing))

0
0
66170


In [57]:
newzcat = quickcat(tile_names, targets, truth, perfect=True)

In [73]:
targets_newzcat = list(newzcat['TARGETID'])
targets_newzcat = list(targets['TARGETID'])

In [74]:
a = list(set(targets_newzcat) & set(targets_fiber))
missing_ids = list(set(targets_fiber) - set(a))
len(missing_ids)
print(missing_ids[:10])

[2631651276564051987, 751670505785917446, 3945118423605182471, 3102139768365806493, 701409607242918572, 3484485554263556106, 36172407562829835, 49281014080471055, 3637729872174645264, 1007073708322062356]


In [62]:
def myquickcat(tilefiles, targets, truth, zcat=None, perfect=False):
    """
    Generates quick output zcatalog
    
    Args:
        tilefiles : list of fiberassign tile files that were observed
        targets : astropy Table of targets
        truth : astropy Table of input truth with columns TARGETID, TRUEZ, and TRUETYPE
        
    Options:
        zcat : input zcatalog Table from previous observations
        perfect : if True, treat spectro pipeline as perfect with input=output,
            otherwise add noise and zwarn!=0 flags
        
    Returns:
        zcatalog astropy Table based upon input truth, plus ZERR, ZWARN,
        NUMOBS, and TYPE columns   
        
    TODO: Add BGS, MWS support     
    """
    #- convert to Table for easier manipulation
    truth = Table(truth)
    
    #- Count how many times each target was observed for this set of tiles
    ### print('Reading {} tiles'.format(len(obstiles)))
    nobs = Counter()
    for infile in tilefiles:
        fibassign = fits.getdata(infile, 'FIBER_ASSIGNMENTS')
        ii = (fibassign['TARGETID'] != -1)  #- targets with assignments
        nobs.update(fibassign['TARGETID'][ii])

        #- Count how many times each target was observed in previous zcatalog
    #- NOTE: assumes that no tiles have been repeated
    if zcat is not None:
        ### print('Counting targets from previous zobs')
        for targetid, n in zip(zcat['TARGETID'], zcat['NUMOBS']):
            nobs[targetid] += n #JF I put n instead of 1

    #- Trim truth down to just ones that have already been observed
    ### print('Trimming truth to just observed targets')
    obs_targetids = np.array(nobs.keys())
    truth_targetids = truth['TARGETID']
    
    a = list(set(truth_targetids) & set(nobs.keys()))
    missing_ids = list(set(nobs.keys()) - set(a))
    if len(missing_ids):
        print('missing ids {}'.format(len(missing_ids)))
        raise NameError('Some IDs in fiberassign are missing from the truth file')
   
    
    iiobs = np.in1d(truth['TARGETID'], obs_targetids)
    print('total {}, intersec {}'.format(len(nobs.keys()), len(truth[iiobs])))
    truth = truth[iiobs]
    targets = targets[iiobs]

    #- Construct initial new z catalog
    newzcat = Table()
    newzcat['TARGETID'] = truth['TARGETID']
    if 'BRICKNAME' in truth.dtype.names:
        newzcat['BRICKNAME'] = truth['BRICKNAME']
    else:
        newzcat['BRICKNAME'] = np.zeros(len(truth), dtype='S8')

    #- Copy TRUEZ -> Z so that we can add errors without altering original
    newzcat['Z'] = truth['TRUEZ'].copy()
    newzcat['TYPE'] = truth['TRUETYPE'].copy()

    #- Add numobs column
    ### print('Adding NUMOBS column')
    nz = len(newzcat)
    newzcat.add_column(Column(name='NUMOBS', length=nz, dtype=np.int32))
    for i in range(nz):
        newzcat['NUMOBS'][i] = nobs[newzcat['TARGETID'][i]]

    #- Add ZERR and ZWARN
    ### print('Adding ZERR and ZWARN')
    if not perfect:
        #- GALAXY -> ELG or LRG
        objtype = newzcat['TYPE'].copy()
        isLRG = (objtype == 'GALAXY') & ((targets['DESI_TARGET'] & desi_mask.LRG) != 0)
        isELG = (objtype == 'GALAXY') & ((targets['DESI_TARGET'] & desi_mask.ELG) != 0)
        objtype[isLRG] = 'LRG'
        objtype[isELG] = 'ELG'
        
        z, zerr, zwarn = get_observed_redshifts(objtype, newzcat['Z'])
        newzcat['Z'] = z  #- update with noisy redshift
    else:
        zerr = np.zeros(nz, dtype=np.float32)
        zwarn = np.zeros(nz, dtype=np.int32)

    newzcat['ZERR'] = zerr
    newzcat['ZWARN'] = zwarn

    #- Metadata for header
    newzcat.meta['EXTNAME'] = 'ZCATALOG'

    return newzcat



In [63]:
mynewzcat = myquickcat(tile_names, targets, truth, perfect=True)

missing ids 66170


NameError: Some IDs in fiberassign are missing from the truth file

In [25]:
targets_mynewzcat = list(mynewzcat['TARGETID'])
a = list(set(targets_mynewzcat) & set(targets_fiber))
missing_ids = list(set(targets_fiber) - set(a))
len(missing_ids)

65329